Copyright 2020 Montvieux Ltd

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


In [ ]:
import requests
from io import BytesIO
import PIL.Image
from IPython.display import display,clear_output,HTML
from IPython.display import Image as DisplayImage
import base64
import json
from io import StringIO
import ipywidgets as widgets
import sys
from plark_game import classes
import time
import imageio
import numpy as np
import matplotlib.pyplot as plt
import io
import os, sys
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.evaluation import evaluate_policy
from gym_plark.envs import plark_env,plark_env_guided_reward,plark_env_top_left

# %matplotlib inline
# %load_ext autoreload
# %autoreload 2
import datetime
basepath = '/data/agents/models'

from stable_baselines import DQN, PPO2, A2C, ACKTR
from stable_baselines.bench import Monitor
from stable_baselines.common.vec_env import DummyVecEnv

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)


import helper 
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))



In [ ]:
env = plark_env_guided_reward.PlarkEnvGuidedReward()


In [ ]:
# It will check your custom environment and output additional warnings if needed
check_env(env)


## Define the player type we are training.
modelplayer = "pelican"
## Define the type of RL algorithm you are using.
modeltype = "dqn"
## Specify the date and time for this training.
basicdate = str(datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))


## Used for generating the json header file which holds details regarding the model.
## This will be used when playing the game from the GUI.


# Instantiate the env
env = plark_env_guided_reward.PlarkEnvGuidedReward()

model = DQN('CnnPolicy', env)
#model = A2C('CnnPolicy', env)
model.learn(500)

retrain_iter = []
retrain_values = []

print("****** STARTING EVALUATION *******")
mean_reward, n_steps = evaluate_policy(model, env, n_eval_episodes=5, deterministic=False, render=False, callback=None, reward_threshold=None, return_episode_rewards=False)
retrain_iter.append(str(0))
retrain_values.append(mean_reward)

def save():
    print(str(retrain_iter))
    print(str(retrain_values))
    
    plt.figure(figsize=(9, 3))
    plt.subplot(131)
    plt.bar(retrain_iter, retrain_values)
    plt.subplot(132)
    plt.scatter(retrain_iter, retrain_values)
    plt.subplot(133)
    plt.plot(retrain_iter, retrain_values)
    plt.suptitle('Retraining Progress')
    ##plt.show()
    model_path,model_dir, modellabel = save_model_with_env_settings(basepath,model,modeltype,env,basicdate)
    fig_path = os.path.join(model_dir, 'Training_Progress.png')
    plt.savefig(fig_path)
    print('Model saved to ', model_path)


def retrain(mean_reward, target_reward, count):
    while mean_reward < target_reward:
        count = count + 1
        retrain_iter.append(str(count))
        
        model.learn(50)
        
        mean_reward, n_steps = evaluate_policy(model, env, n_eval_episodes=1, deterministic=False, render=False, callback=None, reward_threshold=None, return_episode_rewards=False)
        retrain_values.append(mean_reward)
        
        if mean_reward > target_reward:
           save()
           break
        if mean_reward < target_reward:
            retrain(mean_reward, target_reward, count)
    
    print("Model Training Reached Target Level")

retrain(mean_reward, 10, 1)
